In [4]:
# Import Dependencies
import requests
import pandas as pd
import json
from splinter import Browser
from bs4 import BeautifulSoup as bs
import numpy as np
import time
import pymongo
from pymongo import MongoClient 
import datetime
import math

# Cleaning Data Source 1  & Adding to MongoDB #

In [2]:
# Read Source 1 CSV
million_dollar_disasters_df = pd.read_csv('DataFrames/million_dollar_disasters_df.csv')

In [3]:
# Display DataFrame sample
million_dollar_disasters_df.head(1)

,name,disaster,begDate,endDate,adjCost,unadjCost,deaths
0,Southern Severe Storms and Flooding (April 1980),Flooding,19800410,19800417,2728.1,706.8,7


In [4]:
# Display Data Types
million_dollar_disasters_df.dtypes

name          object
disaster      object
begDate        int64
endDate        int64
adjCost      float64
unadjCost    float64
deaths         int64
dtype: object

In [5]:
# Convert begDate and endDate to dates
million_dollar_disasters_df['begDate']=  pd.to_datetime((million_dollar_disasters_df['begDate']), yearfirst=True,format="%Y%m%d")
million_dollar_disasters_df['endDate']=  pd.to_datetime((million_dollar_disasters_df['endDate']), yearfirst=True,format="%Y%m%d")

In [6]:
# Split name into two chunks
million_dollar_disasters_df[['name', 'date']] = million_dollar_disasters_df['name'].str.split('(', n=1, expand=True)

In [7]:
# Remove closing parenthesis on date
million_dollar_disasters_df['date'] = million_dollar_disasters_df['date'].str.replace(")","")

In [8]:
# Truncate adjCost and unadjCost
million_dollar_disasters_df['adjCost'] = [math.trunc(x) for x in million_dollar_disasters_df['adjCost']]
million_dollar_disasters_df['unadjCost'] = [math.trunc(x) for x in million_dollar_disasters_df['unadjCost']]

In [9]:
# Trim extra spaces from name and disaster
million_dollar_disasters_df['name'] = million_dollar_disasters_df['name'].str.strip()
million_dollar_disasters_df['disaster'] = million_dollar_disasters_df['disaster'].str.strip()

In [10]:
# Rename Columns
million_dollar_disasters_df = million_dollar_disasters_df.rename(columns={'name': 'disaster_name','disaster':'disaster_type','begDate':'start_date',
'endDate': 'end_date','adjCost': 'adjusted_cost','unadjCost': 'unadjusted_cost','date':'date category'})

In [11]:
# Review cleaned DataFrame
million_dollar_disasters_df

,disaster_name,disaster_type,start_date,end_date,adjusted_cost,unadjusted_cost,deaths,date category
0,Southern Severe Storms and Flooding,Flooding,1980-04-10,1980-04-17,2728,706,7,April 1980
1,Hurricane Allen,Tropical Cyclone,1980-08-07,1980-08-11,2212,590,13,August 1980
2,Central/Eastern Drought/Heat Wave,Drought,1980-06-01,1980-11-30,40180,10020,1260,Summer-Fall 1980
3,Florida Freeze,Freeze,1981-01-12,1981-01-14,2053,572,0,January 1981
4,"Severe Storms, Flash Floods, Hail, Tornadoes",Severe Storm,1981-05-05,1981-05-10,1397,401,20,May 1981
...,...,...,...,...,...,...,...,...
378,Central and Southern Winter Storm and Cold Wave,Winter Storm,2024-01-14,2024-01-18,1345,1332,41,January 2024
379,Southern Severe Weather,Severe Storm,2024-02-10,2024-02-12,1150,1139,0,February 2024
380,Central and Southern Severe Weather,Severe Storm,2024-03-13,2024-03-15,4354,4311,3,March 2024
381,Central Tornado Outbreak and Eastern Severe We...,Severe Storm,2024-04-01,2024-04-03,1823,1823,3,April 2024


In [12]:
# Create MongoDB database & collection
client = pymongo.MongoClient("mongodb://localhost:27017/")
db = client["million_dollar_disasters"]
collection = db["disasters"]

In [13]:
# Insert data into MongoDB
data_dict = million_dollar_disasters_df.to_dict(orient='records')
collection.insert_many(data_dict)

InsertManyResult([ObjectId('666609fa5105a0a68d1b2ecf'), ObjectId('666609fa5105a0a68d1b2ed0'), ObjectId('666609fa5105a0a68d1b2ed1'), ObjectId('666609fa5105a0a68d1b2ed2'), ObjectId('666609fa5105a0a68d1b2ed3'), ObjectId('666609fa5105a0a68d1b2ed4'), ObjectId('666609fa5105a0a68d1b2ed5'), ObjectId('666609fa5105a0a68d1b2ed6'), ObjectId('666609fa5105a0a68d1b2ed7'), ObjectId('666609fa5105a0a68d1b2ed8'), ObjectId('666609fa5105a0a68d1b2ed9'), ObjectId('666609fa5105a0a68d1b2eda'), ObjectId('666609fa5105a0a68d1b2edb'), ObjectId('666609fa5105a0a68d1b2edc'), ObjectId('666609fa5105a0a68d1b2edd'), ObjectId('666609fa5105a0a68d1b2ede'), ObjectId('666609fa5105a0a68d1b2edf'), ObjectId('666609fa5105a0a68d1b2ee0'), ObjectId('666609fa5105a0a68d1b2ee1'), ObjectId('666609fa5105a0a68d1b2ee2'), ObjectId('666609fa5105a0a68d1b2ee3'), ObjectId('666609fa5105a0a68d1b2ee4'), ObjectId('666609fa5105a0a68d1b2ee5'), ObjectId('666609fa5105a0a68d1b2ee6'), ObjectId('666609fa5105a0a68d1b2ee7'), ObjectId('666609fa5105a0a68d1b2e

# Remember...Source 2 was skipped. # 

# Cleaning Data Source 3  & Adding to MongoDB #

In [16]:
# Read Source 1 CSV
air_pollution_df = pd.read_csv('Dataframes/air_pollution_df.csv')

In [17]:
# Display DataFrame sample
air_pollution_df.head(1)

,measurements_unit,measurements_value,measurements_sourcename,measurements_lastupdated,measurements_parameter,country_name_en
0,ppm,0.037,AirNow,2023-05-30T18:00:00-05:00,O3,United States


In [18]:
# Check data types
air_pollution_df.dtypes

measurements_unit            object
measurements_value          float64
measurements_sourcename      object
measurements_lastupdated     object
measurements_parameter       object
country_name_en              object
dtype: object

In [19]:
# Change measurements_lastupdated to a datetime
air_pollution_df['measurements_lastupdated'] = pd.to_datetime(air_pollution_df['measurements_lastupdated'],yearfirst=True,format="ISO8601")

In [20]:
# Change Column names
air_pollution_df = air_pollution_df.rename(columns={'measurements_value':'measurement_in_ppm','measurements_sourcename': 'source',
'measurements_lastupdated':'last_update','measurements_parameter':'parameter','country_name_en': 'country'})

In [21]:
# Delete measurements_units since we changed the column name to reflect the units
del air_pollution_df['measurements_unit']

In [22]:
# Rearrange the DataFrame columns
air_pollution_df = air_pollution_df[['source','country','parameter','measurement_in_ppm','last_update']]

In [23]:
# Review changes
air_pollution_df.head(1)

,source,country,parameter,measurement_in_ppm,last_update
0,AirNow,United States,O3,0.037,2023-05-30 18:00:00-05:00


In [24]:
# Review data types
air_pollution_df.dtypes

source                                   object
country                                  object
parameter                                object
measurement_in_ppm                      float64
last_update           datetime64[ns, UTC-05:00]
dtype: object

In [25]:
# Create MongoDB database & collection
db = client["air_pollution"]
collection = db["pollution"]

In [26]:
# Insert data into MongoDB
data_dict_1 = air_pollution_df.to_dict(orient='records')
collection.insert_many(data_dict_1)

InsertManyResult([ObjectId('66660a1e5105a0a68d1b304e'), ObjectId('66660a1e5105a0a68d1b304f'), ObjectId('66660a1e5105a0a68d1b3050'), ObjectId('66660a1e5105a0a68d1b3051'), ObjectId('66660a1e5105a0a68d1b3052'), ObjectId('66660a1e5105a0a68d1b3053'), ObjectId('66660a1e5105a0a68d1b3054'), ObjectId('66660a1e5105a0a68d1b3055'), ObjectId('66660a1e5105a0a68d1b3056'), ObjectId('66660a1e5105a0a68d1b3057'), ObjectId('66660a1e5105a0a68d1b3058'), ObjectId('66660a1e5105a0a68d1b3059'), ObjectId('66660a1e5105a0a68d1b305a'), ObjectId('66660a1e5105a0a68d1b305b'), ObjectId('66660a1e5105a0a68d1b305c'), ObjectId('66660a1e5105a0a68d1b305d'), ObjectId('66660a1e5105a0a68d1b305e'), ObjectId('66660a1e5105a0a68d1b305f'), ObjectId('66660a1e5105a0a68d1b3060'), ObjectId('66660a1e5105a0a68d1b3061'), ObjectId('66660a1e5105a0a68d1b3062'), ObjectId('66660a1e5105a0a68d1b3063'), ObjectId('66660a1e5105a0a68d1b3064'), ObjectId('66660a1e5105a0a68d1b3065'), ObjectId('66660a1e5105a0a68d1b3066'), ObjectId('66660a1e5105a0a68d1b30

In [27]:
# Create clean csv to import data into SQL
air_pollution_df.to_csv('DataFrames/ready_for_sql_air_pollution_df.csv', index=False)